# LOGISTIC REGRESSION USING TENSORFLOW

https://builtin.com/data-science/guide-logistic-regression-tensorflow-20

In [14]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf

import numpy as np

## STEP 2: LOADING AND PREPARING THE MNIST DATA SET

In [15]:
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Convert to float32.
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)

num_features=784
# Flatten images to 1-D vector of 784 features (28*28).
x_train, x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])

# Normalize images value from [0, 255] to [0, 1].
x_train, x_test = x_train / 255., x_test / 255.

## SETTING UP HYPERPARAMETERS AND DATA SET PARAMETERS

In [16]:
# MNIST dataset parameters.
num_classes = 10 # 0 to 9 digits

# Training parameters.
learning_rate = 0.01
training_steps = 1000
batch_size = 256
display_step = 50

## STEP 4: SHUFFLING AND BATCHING THE DATA

In [17]:
# Use tf.data API to shuffle and batch data.
train_data=tf.data.Dataset.from_tensor_slices((x_train,y_train))
train_data=train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

## STEP 5: INITIALIZING WEIGHTS AND BIASES

In [18]:
# Weight of shape [784, 10], the 28*28 image features, and a total number of classes.
W = tf.Variable(tf.ones([num_features, num_classes]), name="weight")

# Bias of shape [10], the total number of classes.
b = tf.Variable(tf.zeros([num_classes]), name="bias")

## STEP 6: DEFINING LOGISTIC REGRESSION AND COST FUNCTION

In [19]:
# Logistic regression (Wx + b).
def logistic_regression(x):
    # Apply softmax to normalize the logits to a probability distribution.
    return tf.nn.softmax(tf.matmul(x, W) + b)
# Cross-Entropy loss function.

def cross_entropy(y_pred, y_true):
    # Encode label to a one hot vector.
    y_true = tf.one_hot(y_true, depth=num_classes)
    # Clip prediction values to avoid log(0) error.
    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)
    # Compute cross-entropy.
    return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred)))

## STEP 7: DEFINING OPTIMIZERS AND ACCURACY METRICS

In [20]:
# Accuracy metric.
def accuracy(y_pred, y_true):
    # Predicted class is the index of the highest score in prediction vector (i.e. argmax).
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Stochastic gradient descent optimizer.
optimizer = tf.optimizers.SGD(learning_rate)

## STEP 8: OPTIMIZATION PROCESS AND UPDATING WEIGHTS AND BIASES

In [21]:
# Optimization process. 
def run_optimization(x, y):
# Wrap computation inside a GradientTape for automatic differentiation.
    with tf.GradientTape() as g:
        pred = logistic_regression(x)
        loss = cross_entropy(pred, y)

    # Compute gradients.
    gradients = g.gradient(loss, [W, b])

    # Update W and b following gradients.
    optimizer.apply_gradients(zip(gradients, [W, b]))

## STEP 9: THE TRAINING LOOP

In [22]:
# Run training for the given number of steps.
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
    # Run the optimization to update W and b values.
    run_optimization(batch_x, batch_y)

    if step % display_step == 0:
        pred = logistic_regression(batch_x)
        loss = cross_entropy(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))

step: 50, loss: 1020.792236, accuracy: 0.753906
step: 100, loss: 1270.986450, accuracy: 0.703125
step: 150, loss: 1100.225708, accuracy: 0.757812
step: 200, loss: 1274.216553, accuracy: 0.707031
step: 250, loss: 1018.189392, accuracy: 0.769531
step: 300, loss: 1293.071533, accuracy: 0.718750
step: 350, loss: 1079.716064, accuracy: 0.777344
step: 400, loss: 1243.091797, accuracy: 0.738281
step: 450, loss: 1079.991455, accuracy: 0.738281
step: 500, loss: 1055.208984, accuracy: 0.789062
step: 550, loss: 1036.540649, accuracy: 0.785156
step: 600, loss: 1182.085083, accuracy: 0.734375
step: 650, loss: 1146.489868, accuracy: 0.753906
step: 700, loss: 1052.564941, accuracy: 0.757812
step: 750, loss: 1193.673340, accuracy: 0.734375
step: 800, loss: 975.707336, accuracy: 0.800781
step: 850, loss: 1268.664062, accuracy: 0.710938
step: 900, loss: 1036.599609, accuracy: 0.789062
step: 950, loss: 727.342529, accuracy: 0.734375
step: 1000, loss: 633.336731, accuracy: 0.847656


## STEP 10: TESTING MODEL ACCURACY USING THE TEST DATA

In [23]:
# Test model on validation set.
pred = logistic_regression(x_test)
print("Test Accuracy: %f" % accuracy(pred, y_test))

Test Accuracy: 0.836300
